# 🛡️ Citadel Agent Access Contract Request - Foundry SDK Validation

This notebook validates the **Agent Access Contract Request** system end-to-end:

1. **Define** an Access Contract Request JSON with Foundry connection enabled
2. **Deploy** the contract using `az deployment sub create` with the `base-access-contract-request/main.bicep` module
3. **Verify** deployment outputs (products, subscriptions, Foundry connections, generated policy XML)
4. **Retrieve** APIM subscription keys for the deployed contract
5. **Build** a Microsoft Foundry SDK agent using the deployed access contract
6. **Test** the agent with a multi-turn conversation routed through Citadel
7. **Analyze** token consumption, retries, and performance metrics

### Prerequisites
- Azure CLI authenticated (`az login`)
- Citadel Governance Hub deployed (APIM + backend pools)
- Azure AI Foundry account + project provisioned
- Python 3.10+ with packages: `azure-identity`, `azure-ai-projects`, `azure-mgmt-apimanagement`

### Integration Pattern
```
Foundry SDK Agent → connection_name/model_name → Citadel APIM → Backend Pool → Azure OpenAI
```

---
## 🔧 Configuration
---

<a id='1'></a>
### 1️⃣ Initialize Variables

Configure the Citadel Governance Hub, Azure AI Foundry, and deployment settings.

In [ ]:
import sys, os, json, time, requests, importlib
sys.path.insert(1, '../shared')
import utils
importlib.reload(utils)  # Reload to pick up any changes to utils.py
from apimtools import APIMClientTool

# ============================================================================
# CITADEL GOVERNANCE HUB CONFIGURATION
# ============================================================================
governance_hub_resource_group = "REPLACE"  # Your Citadel APIM resource group
targetInferenceApi = "models"  # use 'models' for universal LLM API, or 'openai' for Azure OpenAI

# ============================================================================
# AZURE AI FOUNDRY CONFIGURATION
# ============================================================================
foundry_account_name     = "REPLACE"    # AI Foundry account name
foundry_project_name     = "crm-support-agent"             # AI Foundry project name
foundry_resource_group   = "REPLACE"        # Foundry resource group
foundry_connection_name  = "REPLACE"         # Connection name for Citadel routing

# ============================================================================
# MODEL CONFIGURATION
# ============================================================================
model_name       = "gpt-4o"                                 # Primary model
api_version      = "2024-12-01-preview"                     # Azure OpenAI API version

# ============================================================================
# DEPLOYMENT CONFIGURATION
# ============================================================================
deployment_name     = "agent-access-contract-test"          # Bicep deployment name
deployment_location = "swedencentral"                       # Deployment location

# ============================================================================
# RETRY CONFIGURATION
# ============================================================================
MAX_RETRIES = 3
RETRY_DELAY_BASE = 5  # seconds

# ============================================================================
# METRICS TRACKING
# ============================================================================
agent_metrics = {}

utils.print_ok("Configuration initialized")
utils.print_info(f"Governance Hub RG: {governance_hub_resource_group}")
utils.print_info(f"Foundry Account:   {foundry_account_name}")
utils.print_info(f"Foundry Project:   {foundry_project_name}")
utils.print_info(f"Connection Name:   {foundry_connection_name}")
utils.print_info(f"Model:             {model_name}")

<a id='2'></a>
### 2️⃣ Verify Azure CLI & Initialize APIM Client

Ensure Azure CLI is authenticated and APIM client is ready.

In [ ]:
# Verify Azure CLI authentication
output = utils.run("az account show", "Azure CLI authenticated", "Azure CLI not authenticated. Run 'az login' first.")

if output.success and output.json_data:
    subscription_id = output.json_data['id']
    subscription_name = output.json_data['name']
    tenant_id = output.json_data['tenantId']
    utils.print_info(f"Subscription: {subscription_id} ({subscription_name})")
    utils.print_info(f"Tenant: {tenant_id}")
else:
    raise Exception("Azure CLI is not authenticated. Please run 'az login'.")

# Initialize APIM Client
apim_client = APIMClientTool(governance_hub_resource_group)
apim_client.initialize()

# Attempt API discovery (optional - Foundry SDK agent uses connection routing, not direct APIM endpoint)
chat_completions_url = None
try:
    apim_client.discover_api(targetInferenceApi)
    chat_completions_url = f"{apim_client.azure_endpoint}/openai/deployments/{model_name}/chat/completions?api-version={api_version}"
    utils.print_ok(f"APIM Endpoint: {apim_client.azure_endpoint}")
    utils.print_info(f"Chat Completions URL: {chat_completions_url}")
except Exception as e:
    utils.print_warning(f"API discovery skipped: {e}")
    utils.print_info("This is OK — Foundry SDK agent uses connection_name/model_name routing, not direct APIM endpoint.")
    # List available APIs for debugging
    try:
        apis = apim_client.client.api.list_by_service(governance_hub_resource_group, apim_client.apim_resource_name)
        utils.print_info("Available APIs in APIM:")
        for api in apis:
            print(f"  📡 {api.display_name} — path: {api.path}")
    except Exception:
        pass

utils.print_ok(f"APIM Client initialized: {apim_client.apim_resource_name}")

---
## 📝 Agent Access Contract Request
---

<a id='3'></a>
### 3️⃣ Define the Access Contract Request

Create a Foundry-connected access contract request with:
- **Foundry connection** enabled for Foundry SDK agent routing
- **Model access** restricted to `gpt-4o`
- **Capacity management** at subscription level
- **Content safety** with Hate and Violence categories
- **Usage tracking** with custom dimensions for agent tracing
- **Throttling alerts** for monitoring

This JSON replaces the need for separate `.bicepparam` + `.xml` files.

In [ ]:
# Define the Agent Access Contract Request with Foundry connection
access_contract_request = {
    "$schema": "../../access-contract-request.schema.json",
    "contractInfo": {
        "businessUnit": "CRM",
        "useCaseName": "SupportAgent",
        "environment": "DEV",
        "description": "CRM Support Agent built with Microsoft Foundry SDK - routes through Citadel via Foundry connection for governed LLM access.",
        "productTerms": "CRM department AI agent for customer support. All interactions are routed through Citadel Governance Hub."
    },
    "infrastructure": {
        "apim": {
            "subscriptionId": subscription_id,
            "resourceGroupName": governance_hub_resource_group,
            "name": apim_client.apim_resource_name
        },
        "keyVault": {
            "enabled": False
        },
        "foundry": {
            "enabled": True,
            "subscriptionId": subscription_id,
            "resourceGroupName": foundry_resource_group,
            "accountName": foundry_account_name,
            "projectName": foundry_project_name
        }
    },
    "apiNameMapping": {
        "LLM": ["universal-llm-api", "azure-openai-api"]
    },
    "services": [
        {
            "code": "LLM",
            "endpointSecretName": f"CRM-SUPPORTAGENT-LLM-ENDPOINT",
            "apiKeySecretName": f"CRM-SUPPORTAGENT-LLM-KEY"
        }
    ],
    "policies": {
        "modelAccess": {
            "enabled": True,
            "allowedModels": [model_name]
        },
        "capacityManagement": {
            "enabled": True,
            "mode": "subscription-level",
            "subscriptionLevel": {
                "tokensPerMinute": 5000,
                "tokenQuota": 500000,
                "tokenQuotaPeriod": "Monthly"
            }
        },
        "contentSafety": {
            "enabled": True,
            "shieldPrompt": True,
            "categories": [
                {"name": "Hate", "threshold": 4},
                {"name": "Violence", "threshold": 4}
            ]
        },
        "usageTracking": {
            "enabled": True,
            "appIdHeader": "x-app-id",
            "customDimension1": {
                "header": "x-agent-id",
                "default": "crm-support-agent"
            },
            "customDimension2": {
                "header": "x-session-id",
                "default": "unknown-session"
            }
        },
        "alerts": {
            "enabled": True,
            "throttlingEvents": True
        }
    }
}

# Display the contract request
utils.print_ok("Access Contract Request defined")
print(json.dumps(access_contract_request, indent=2))

# Save to temp file for Bicep deployment
contract_dir = os.path.join(os.getcwd(), '.temp')
os.makedirs(contract_dir, exist_ok=True)

contract_file = os.path.join(contract_dir, 'access-contract-request.json')
with open(contract_file, 'w') as f:
    json.dump(access_contract_request, f, indent=2)

utils.print_ok(f"Contract request saved to: {contract_file}")

<a id='4'></a>
### 4️⃣ Generate Bicep Parameters File

Create a `.bicepparam` file that references the access contract request JSON.
The Bicep module will load the JSON and automatically generate the APIM policy XML.

In [ ]:
# Generate the .bicepparam file that references the JSON
bicepparam_content = """using '../../bicep/infra/citadel-access-contracts/base-access-contract-request/main.bicep'

// ============================================================================
// CRM Support Agent - Agent Access Contract Request (Foundry SDK)
// ============================================================================
// Auto-generated by validation notebook.
// This contract enables Foundry SDK agent routing through Citadel.
// ============================================================================

param contractRequest = loadJsonContent('access-contract-request.json')
"""

bicepparam_file = os.path.join(contract_dir, 'main.bicepparam')
with open(bicepparam_file, 'w') as f:
    f.write(bicepparam_content)

utils.print_ok(f"Bicepparam file saved to: {bicepparam_file}")
utils.print_info(f"Contract request JSON: {contract_file}")
print("\n" + bicepparam_content)

---
## 🚀 Deploy Access Contract
---

<a id='5'></a>
### 5️⃣ Deploy the Access Contract via Bicep

Deploy the Agent Access Contract Request using `az deployment sub create`.
The Bicep module will:
1. Read the JSON configuration
2. Generate the APIM policy XML from snippet templates
3. Create the APIM product, subscription, and policy
4. Set up the Foundry connection for the agent

In [ ]:
# Deploy the access contract using Bicep
utils.print_info(f"🚀 Deploying Access Contract: CRM-SupportAgent-DEV")
utils.print_info(f"   Deployment: {deployment_name}")
utils.print_info(f"   Location:   {deployment_location}")
utils.print_info(f"   Template:   base-access-contract-request/main.bicep")
utils.print_info(f"   Parameters: {bicepparam_file}")

deploy_output = utils.run(
    f'az deployment sub create '
    f'--name {deployment_name} '
    f'--location {deployment_location} '
    f'--parameters "{bicepparam_file}" '
    f'-o json',
    "✅ Access Contract deployed successfully",
    "❌ Access Contract deployment failed"
)

if deploy_output.success and deploy_output.json_data:
    provisioning_state = deploy_output.json_data.get('properties', {}).get('provisioningState', 'Unknown')
    utils.print_ok(f"Provisioning State: {provisioning_state}")

    # Extract Foundry connection name and model from deployment outputs
    deploy_outputs = deploy_output.json_data.get('properties', {}).get('outputs', {})
    deployed_foundry_connections = deploy_outputs.get('foundryConnections', {}).get('value', [])
    if deployed_foundry_connections:
        # Use the first LLM connection (matching service code "LLM")
        for fc in deployed_foundry_connections:
            if fc.get('code', '').upper() == 'LLM':
                foundry_connection_name = fc['connectionName']
                break
        else:
            # Fallback to first connection if no LLM match
            foundry_connection_name = deployed_foundry_connections[0]['connectionName']
        utils.print_ok(f"Foundry Connection Name (from deployment): {foundry_connection_name}")
        utils.print_info(f"Agent deployment target: {foundry_connection_name}/{model_name}")
    else:
        utils.print_warning(f"No Foundry connections in deployment output. Using configured value: {foundry_connection_name}")
else:
    utils.print_error("Deployment failed. Check the output above for details.")
    utils.print_info("Tip: Ensure the Citadel Governance Hub is deployed and the APIM resource exists.")

<a id='6'></a>
### 6️⃣ Verify Deployment Outputs

Inspect the deployment outputs including:
- APIM Gateway URL
- Created products and subscriptions
- Foundry connections
- Generated APIM policy XML

In [ ]:
# print(deploy_output.text)
# print(deploy_output.json_data)
if deploy_output.success and deploy_output.json_data:
    outputs = deploy_output.json_data.get('properties', {}).get('outputs', {})
    
    # APIM Gateway URL
    apim_gateway = outputs.get('apimGatewayUrl', {}).get('value', 'N/A')
    utils.print_ok(f"APIM Gateway URL: {apim_gateway}")
    
    # Key Vault integration
    use_kv = outputs.get('useKeyVault', {}).get('value', False)
    utils.print_info(f"Key Vault Enabled: {use_kv}")
    
    # Foundry integration
    use_foundry = outputs.get('useFoundry', {}).get('value', False)
    foundry_connections = outputs.get('foundryConnections', {}).get('value', [])
    utils.print_info(f"Foundry Enabled: {use_foundry}")
    if foundry_connections:
        utils.print_ok(f"Foundry Connections: {json.dumps(foundry_connections, indent=2)}")
    
    # Products
    products = outputs.get('products', {}).get('value', [])
    utils.print_ok(f"Products created: {len(products)}")
    for p in products:
        print(f"  📦 {p}")
    
    # Subscriptions
    subscriptions = outputs.get('subscriptions', {}).get('value', [])
    utils.print_ok(f"Subscriptions created: {len(subscriptions)}")
    for s in subscriptions:
        print(f"  🔑 {s}")
    
    # Endpoints
    endpoints = outputs.get('endpoints', {}).get('value', [])
    utils.print_ok(f"Endpoints: {len(endpoints)}")
    for ep in endpoints:
        masked = utils.mask_sensitive_values(ep) if isinstance(ep, dict) else ep
        print(f"  🌐 {masked}")
    
    # Generated Policy XML
    policy_xml = outputs.get('generatedPolicyXml', {}).get('value', '')
    if policy_xml:
        utils.print_ok("Generated APIM Policy XML:")
        print("=" * 80)
        print(policy_xml[:3000])  # Truncate for display
        if len(policy_xml) > 3000:
            print(f"\n... (truncated, total length: {len(policy_xml)} chars)")
        print("=" * 80)
else:
    utils.print_warning("No deployment outputs available. Using existing APIM configuration.")

---
## 🔑 Retrieve Access Contract Keys
---

<a id='7'></a>
### 7️⃣ Retrieve APIM Subscription Keys for Access Contract

Find the APIM subscription key for the deployed CRM-SupportAgent access contract.

In [ ]:
# Expected product ID from the access contract naming convention:
# {businessUnit}-{useCaseName} = CRM-SupportAgent
expected_product_id = "CRM-SupportAgent"

# Refresh APIM subscriptions after deployment
apim_client_refreshed = APIMClientTool(governance_hub_resource_group)
apim_client_refreshed.initialize()

contract_key = None
for sub in apim_client_refreshed.apim_subscriptions:
    if expected_product_id.lower() in sub['name'].lower():
        contract_key = sub['key']
        utils.print_ok(f"Found subscription key for '{sub['name']}'")
        utils.print_info(f"Key: ****{contract_key[-4:]}")
        break

if not contract_key:
    utils.print_warning(f"Subscription key for '{expected_product_id}' not found.")
    utils.print_info("Available subscriptions:")
    for sub in apim_client_refreshed.apim_subscriptions:
        print(f"  🔑 {sub['name']}")
    utils.print_info("\nPlease select the correct subscription name above and set contract_key manually.")

---
## 📦 Install Dependencies
---

<a id='8'></a>
### 8️⃣ Install Foundry SDK Packages

Install the Microsoft Foundry SDK and dependencies for building agents.

In [ ]:
%pip install azure-ai-projects>=2.0.0b2 azure-identity nest-asyncio --quiet

---
## 🤖 Build Foundry SDK Agent
---

<a id='9'></a>
### 9️⃣ Define the Foundry CRM Support Agent

Build an AI agent using the Microsoft Foundry SDK with:
- `AIProjectClient` for Foundry project access
- `PromptAgentDefinition` for agent definition
- `connection_name/model_name` deployment pattern for Citadel routing
- Conversations and Responses API for multi-turn chat

**Integration Pattern:**
```
AIProjectClient → create agent with connection_name/model_name
  → Foundry routes to Citadel APIM via connection
    → APIM applies access contract policies (model access, capacity, safety, tracking)
      → Request reaches Azure OpenAI backend pool
```

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from datetime import datetime
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition


class FoundryCRMSupportAgent:
    """
    CRM Support Agent using Microsoft Foundry SDK with Prompt Agent pattern.
    Uses the Agent Access Contract Request's Foundry connection to route
    through Citadel Governance Hub.

    Based on the working FoundryHRAgent pattern from citadel-agent-frameworks-tests.
    """

    def __init__(self, foundry_account: str, foundry_project: str,
                 connection_name: str, model_name: str):
        """
        Initialize Foundry CRM Support Agent.

        Args:
            foundry_account: AI Foundry account name
            foundry_project: AI Foundry project name
            connection_name: Foundry connection name for Citadel routing
            model_name: Model name (routes via connection_name/model_name)
        """
        self.foundry_account = foundry_account
        self.foundry_project = foundry_project
        self.connection_name = connection_name
        self.model_name = model_name
        self.deployment_name = f"{connection_name}/{model_name}"
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.calls = 0
        self.retries = 0
        self.agent = None
        self.project_client = None
        self.conversation_id = None

        # System prompt for CRM Support Agent
        self.system_prompt = (
            f"You are a CRM Support Agent AI assistant. Your role is to:\n"
            f"- Help customer service representatives with CRM-related queries\n"
            f"- Provide guidance on customer account management\n"
            f"- Assist with order tracking, returns, and escalation procedures\n"
            f"- Suggest upselling and retention strategies based on customer history\n"
            f"- Draft professional customer communication templates\n\n"
            f"Be professional, concise, and action-oriented.\n"
            f"Current date: {datetime.now().strftime('%Y-%m-%d')}"
        )

        # Build Foundry endpoint (must include /api/projects/{project})
        self.endpoint = f"https://{foundry_account}.services.ai.azure.com/api/projects/{foundry_project}"

        try:
            utils.print_info(f"Connecting to Foundry endpoint: {self.endpoint}")

            # Initialize AIProjectClient
            self.project_client = AIProjectClient(
                endpoint=self.endpoint,
                credential=DefaultAzureCredential()
            )
            utils.print_ok(f"Foundry project client initialized for: {foundry_project}")

            # Create a versioned Prompt Agent using the official pattern
            self.agent = self.project_client.agents.create_version(
                agent_name="CRM-SupportAgent",
                definition=PromptAgentDefinition(
                    model=self.deployment_name,  # connection_name/model_name routes through Citadel
                    instructions=self.system_prompt
                )
            )

            utils.print_ok(f"Agent created (id: {self.agent.id}, name: {self.agent.name}, version: {self.agent.version})")
            utils.print_info(f"Endpoint:    {self.endpoint}")
            utils.print_info(f"Deployment:  {self.deployment_name}")

        except Exception as e:
            utils.print_error(f"Failed to initialize Foundry agent: {e}")
            raise

    def chat(self, user_message: str) -> str:
        """
        Send a message using Foundry conversations/responses API with retry logic.
        Uses get_openai_client() as context manager per the official SDK pattern.
        """
        last_error = None
        for attempt in range(MAX_RETRIES + 1):
            try:
                return self._chat_with_agent(user_message)
            except Exception as e:
                last_error = e
                if attempt < MAX_RETRIES:
                    self.retries += 1
                    delay = RETRY_DELAY_BASE * (2 ** attempt)
                    utils.print_warning(f"  ⚠️ Attempt {attempt + 1}/{MAX_RETRIES + 1} failed: {e}")
                    utils.print_info(f"  ⏳ Retrying in {delay}s...")
                    time.sleep(delay)
                else:
                    error_msg = (
                        f"❌ Foundry CRM Support Agent call failed after {MAX_RETRIES + 1} attempts.\n"
                        f"  Last error: {last_error}\n"
                        f"  → Check that the Foundry connection is configured correctly\n"
                        f"  → Verify the access contract is deployed and active in APIM\n"
                        f"  → Ensure the model '{self.model_name}' is in the backend pool\n"
                        f"  → Endpoint: {self.endpoint}"
                    )
                    utils.print_error(error_msg)
                    return f"[ERROR] {error_msg}"

    def _chat_with_agent(self, user_message: str) -> str:
        """Chat using the Prompt Agent with conversations and responses API.

        Uses get_openai_client() as context manager, matching the official SDK pattern.
        """
        with self.project_client.get_openai_client() as openai_client:
            if self.conversation_id is None:
                # Create a new conversation with the first user message
                conversation = openai_client.conversations.create(
                    items=[{"type": "message", "role": "user", "content": user_message}],
                )
                self.conversation_id = conversation.id
                utils.print_info(f"Created conversation (id: {self.conversation_id})")
            else:
                # Add user message to existing conversation
                openai_client.conversations.items.create(
                    conversation_id=self.conversation_id,
                    items=[{"type": "message", "role": "user", "content": user_message}],
                )

            # Get response from the agent
            response = openai_client.responses.create(
                conversation=self.conversation_id,
                extra_body={"agent": {"name": self.agent.name, "type": "agent_reference"}},
                input="",
            )

            content = response.output_text if hasattr(response, 'output_text') else str(response)

            # Track metrics - estimate tokens since responses API may not return usage
            estimated_prompt_tokens = len(user_message.split())
            estimated_completion_tokens = int(len(content.split()) * 1.3)
            self.prompt_tokens += estimated_prompt_tokens
            self.completion_tokens += estimated_completion_tokens
            self.total_tokens = self.prompt_tokens + self.completion_tokens
            self.calls += 1

            return content

    def get_metrics(self) -> dict:
        """Return token usage metrics including retries."""
        return {
            "total_tokens": self.total_tokens,
            "prompt_tokens": self.prompt_tokens,
            "completion_tokens": self.completion_tokens,
            "calls": self.calls,
            "retries": self.retries
        }

    def close(self, delete_version: bool = False):
        """Clean up resources.

        Args:
            delete_version: If True, deletes the agent version from Foundry.
        """
        if delete_version:
            try:
                if self.agent and self.project_client:
                    self.project_client.agents.delete_version(
                        agent_name=self.agent.name,
                        agent_version=self.agent.version
                    )
                    utils.print_ok(f"Agent version deleted (name: {self.agent.name}, version: {self.agent.version})")
            except Exception as e:
                utils.print_warning(f"Failed to delete agent version: {e}")
        else:
            if self.agent:
                utils.print_info(f"Agent version retained (name: {self.agent.name}, version: {self.agent.version}). Pass delete_version=True to remove it.")


utils.print_ok("FoundryCRMSupportAgent class defined!")

---
## 💬 Agent Conversation Test
---

<a id='10'></a>
### 🔟 Run Multi-Turn Conversation Test

Simulate a realistic CRM support conversation with the Foundry agent.
Each message routes through: **Foundry → Citadel APIM (policy enforcement) → Azure OpenAI**

In [ ]:
# Resolve the Foundry connection name from deployment output (safety net in case deploy cell wasn't re-run)
if deploy_output.success and deploy_output.json_data:
    _deploy_outputs = deploy_output.json_data.get('properties', {}).get('outputs', {})
    _fc_list = _deploy_outputs.get('foundryConnections', {}).get('value', [])
    for _fc in _fc_list:
        if _fc.get('code', '').upper() == 'LLM':
            foundry_connection_name = _fc['connectionName']
            break

utils.print_info("🤖 Starting Foundry CRM Support Agent conversation...")
utils.print_info("=" * 60)
utils.print_info(f"📡 Using Foundry connection: {foundry_connection_name}/{model_name}")
utils.print_info(f"🛡️  Access Contract: CRM-SupportAgent-DEV")
utils.print_info(f"   Policies: model-access, capacity (5K TPM), content-safety, usage-tracking, alerts")

try:
    crm_agent = FoundryCRMSupportAgent(
        foundry_account=foundry_account_name,
        foundry_project=foundry_project_name,
        connection_name=foundry_connection_name,
        model_name=model_name
    )

    # Simulate a CRM support conversation
    crm_conversation = [
        "A customer is complaining about a delayed order #ORD-2024-8891. Can you help me draft a response?",
        "The order was supposed to arrive 5 days ago. The tracking shows it's stuck at a distribution center. What should I offer?",
        "The customer has been with us for 3 years and has a VIP loyalty tier. Should I offer a discount on their next order?",
        "Can you draft a professional apology email with the compensation offer we discussed?",
        "Great, one more thing - the customer also asked about upgrading to our Premium plan. What are the key benefits I should mention?"
    ]

    for i, user_msg in enumerate(crm_conversation, 1):
        print(f"\n👤 CRM Rep ({i}/{len(crm_conversation)}): {user_msg}")
        response = crm_agent.chat(user_msg)
        print(f"🤖 CRM Agent: {response[:400]}{'...' if len(response) > 400 else ''}")
        time.sleep(1)  # Rate limiting

    # Store metrics
    agent_metrics["Foundry SDK (CRM Support)"] = crm_agent.get_metrics()
    utils.print_ok(f"\n📊 CRM Agent Metrics: {agent_metrics['Foundry SDK (CRM Support)']}")

    # Cleanup
    crm_agent.close()

except Exception as e:
    utils.print_error(f"Failed to run CRM Support Agent: {e}")

---
## 📊 Performance Analysis
---

<a id='11'></a>
### 1️⃣1️⃣ Display Agent Statistics

Visualize token consumption and retry behavior.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

active_agents = {k: v for k, v in agent_metrics.items() if v.get('total_tokens', 0) > 0}

if active_agents:
    # Print summary table
    print("\n" + "=" * 90)
    print("📊 AGENT ACCESS CONTRACT REQUEST - PERFORMANCE SUMMARY")
    print("=" * 90)
    print(f"{'Agent':<35} {'Calls':<8} {'Retries':<10} {'Prompt':<12} {'Completion':<12} {'Total':<12}")
    print("-" * 90)

    total_all_tokens = 0
    total_all_retries = 0
    for agent_name, metrics in active_agents.items():
        retries = metrics.get('retries', 0)
        print(f"{agent_name:<35} {metrics['calls']:<8} {retries:<10} {metrics['prompt_tokens']:<12} {metrics['completion_tokens']:<12} {metrics['total_tokens']:<12}")
        total_all_tokens += metrics['total_tokens']
        total_all_retries += retries

    print("-" * 90)
    print(f"{'TOTAL':<35} {'':<8} {total_all_retries:<10} {'':<12} {'':<12} {total_all_tokens:<12}")
    print("=" * 90)

    # Create visualization
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    colors = ['#2E86AB', '#A23B72', '#F18F01', '#4CAF50']
    labels = list(active_agents.keys())

    # 1. Token Distribution (Stacked Bar)
    x = np.arange(len(labels))
    prompt_tokens = [m['prompt_tokens'] for m in active_agents.values()]
    completion_tokens = [m['completion_tokens'] for m in active_agents.values()]

    axes[0].bar(x, prompt_tokens, label='Prompt Tokens', color='#2E86AB', alpha=0.8)
    axes[0].bar(x, completion_tokens, bottom=prompt_tokens, label='Completion Tokens', color='#F18F01', alpha=0.8)
    axes[0].set_xticks(x)
    axes[0].set_xticklabels([l.split('(')[0].strip() for l in labels], rotation=15, ha='right')
    axes[0].set_ylabel('Tokens')
    axes[0].set_title('Token Distribution', fontsize=12, fontweight='bold')
    axes[0].legend()

    # 2. Calls vs Retries
    calls = [m['calls'] for m in active_agents.values()]
    retries = [m.get('retries', 0) for m in active_agents.values()]

    bar_width = 0.35
    axes[1].bar(x - bar_width / 2, calls, bar_width, label='Successful Calls', color='#2E86AB', alpha=0.8)
    axes[1].bar(x + bar_width / 2, retries, bar_width, label='Retries', color='#E84855', alpha=0.8)
    axes[1].set_xticks(x)
    axes[1].set_xticklabels([l.split('(')[0].strip() for l in labels], rotation=15, ha='right')
    axes[1].set_ylabel('Count')
    axes[1].set_title('Calls vs Retries', fontsize=12, fontweight='bold')
    axes[1].legend()

    # 3. Token Efficiency
    efficiencies = []
    for m in active_agents.values():
        eff = (m['completion_tokens'] / m['total_tokens'] * 100) if m['total_tokens'] > 0 else 0
        efficiencies.append(eff)

    bars = axes[2].barh(labels, efficiencies, color=colors[:len(labels)], alpha=0.8)
    for bar, eff in zip(bars, efficiencies):
        axes[2].text(bar.get_width() + 1, bar.get_y() + bar.get_height() / 2,
                     f'{eff:.1f}%', va='center', fontweight='bold')
    axes[2].set_xlabel('Efficiency (%)')
    axes[2].set_title('Token Efficiency (Completion/Total)', fontsize=12, fontweight='bold')
    axes[2].set_xlim(0, max(efficiencies) * 1.3 if efficiencies else 100)

    plt.tight_layout()
    plt.savefig('agent_access_contract_metrics.png', dpi=150, bbox_inches='tight')
    plt.show()

    utils.print_ok("Visualization saved to 'agent_access_contract_metrics.png'")
else:
    utils.print_warning("No agent metrics available. Run the agent test first.")

---
## 🧹 Cleanup
---

<a id='12'></a>
### 1️⃣2️⃣ Cleanup Temporary Files

Remove the temporary contract request and bicepparam files generated during testing.

In [ ]:
import shutil

# Clean up temp directory
if os.path.exists(contract_dir):
    shutil.rmtree(contract_dir)
    utils.print_ok(f"Cleaned up temporary directory: {contract_dir}")

# Clean up metrics visualization
metrics_file = 'agent_access_contract_metrics.png'
if os.path.exists(metrics_file):
    os.remove(metrics_file)
    utils.print_ok(f"Cleaned up: {metrics_file}")

<a id='summary'></a>
### 📋 Summary

This notebook validated the **Agent Access Contract Request** system:

| Step | Description | What It Proves |
|------|-------------|----------------|
| JSON Definition | Created contract with Foundry connection + policies | Single JSON replaces `.bicepparam` + `.xml` |
| Bicep Deployment | Deployed via `base-access-contract-request/main.bicep` | Bicep auto-generates APIM policy XML |
| Output Verification | Inspected products, subscriptions, Foundry connections, policy | End-to-end deployment pipeline works |
| Foundry SDK Agent | Built agent with `connection_name/model_name` | Foundry SDK integrates with Citadel |
| Conversation Test | 5-turn CRM support dialogue | Policies enforced (model access, capacity, safety, tracking, alerts) |
| Metrics Analysis | Token consumption and retry analysis | Performance visibility across governed access |

**Integration Pattern Used:**
```
Foundry SDK (AIProjectClient)
  → PromptAgentDefinition(model="connection_name/model_name")
  → Conversations + Responses API
  → Citadel APIM (enforces access contract policies)
  → Azure OpenAI backend pool
```

**Key Benefit:** A single `access-contract-request.json` file replaces the need to author:
- Complex `.bicepparam` files with nested infrastructure references
- Handcrafted APIM policy XML with 14+ conditional policy sections
- Manual Foundry connection configuration

In [ ]:
# Final summary
utils.print_info("\n" + "=" * 60)
utils.print_info("🎉 AGENT ACCESS CONTRACT REQUEST TESTING COMPLETE!")
utils.print_info("=" * 60)

total_retries = 0
for agent_name, metrics in agent_metrics.items():
    retries = metrics.get('retries', 0)
    total_retries += retries
    if metrics.get('total_tokens', 0) > 0:
        retry_info = f", {retries} retries" if retries > 0 else ""
        utils.print_ok(f"✅ {agent_name}: {metrics['total_tokens']} total tokens, {metrics['calls']} calls{retry_info}")
    else:
        utils.print_warning(f"⚠️ {agent_name}: No data (agent may not have been tested)")

if total_retries > 0:
    utils.print_warning(f"\n🔄 Total retries: {total_retries}")
    utils.print_info("  Consider increasing capacity limits in the access contract request if retries are frequent")

utils.print_info("\n📌 Next Steps:")
utils.print_info("  1. Review the generated APIM policy XML in deployment outputs")
utils.print_info("  2. Customize the access-contract-request.json for your use case")
utils.print_info("  3. Add PII handling policies if needed (anonymize or block mode)")
utils.print_info("  4. Monitor usage through the custom tracking dimensions")
utils.print_info("  5. Set up Power BI dashboards using the usage tracking data")